# Deduplication Persons

In [2]:
# %load /home/gaetan/Desktop/geovpylib/templates/heading.py
%load_ext autoreload
%autoreload 2

# Common imports
import os
import pandas as pd, numpy as np
import datetime
#import time
#import json
#import requests
#import duckdb
#import plotly.express as px
# from multiprocessing import Pool

# Geovpylib library
import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.decorators as d
import geovpylib.importer as i
import geovpylib.magics
import geovpylib.pks as pks
import geovpylib.queries as q
import geovpylib.record_linkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u
eta = u.Eta()

# Specific imports
# ...

# Global variables
# ...

# Connect to Geovistory database read mode
# db.connect_geovistory('prod')

# Connect to Geovistory database for insert
env = 'prod' # Database to query: "prod", "stag", "dev", "local"
pk_project = pks.projects.switzerland_and_beyond # The project to query/insert: integer
execute = False # Boolean to prevent to execute directly into databases
metadata_str = '' # kebab-lower-case or snake-lower-case. 
import_manner = 'one-shot' # 'one-shot' or 'batch'
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

# Connect to other database
# db_url_env_var_name = 'YELLOW_' # Name of an environment variable holding the Postgres database URL
# execute = False # Boolean to prevent to execute directly into databases
# db.connect_external(os.getenv(db_url_env_var_name), execute=False)

# Connect to a SPARQL endpoint
# sparql.connect_external('url')


[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!


# Fetch data

In [30]:

persons = db.query(f"""
    select
        r0.pk_entity as pk_person,
        a3.string as name
    from information.resource r0
    inner join projects.info_proj_rel ipr0 on ipr0.fk_entity = r0.pk_entity and ipr0.fk_project = {pk_project} and ipr0.is_in_project = true
    inner join information.statement s1 on s1.fk_object_info = r0.pk_entity and s1.fk_property = {pks.properties.aial_isAppelationForLanguageOf_entity}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = {pks.properties.aial_refersToName_appellation}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
    where r0.fk_class = {pks.classes.person}
""")

births = db.query(f"""
    select
        s1.fk_object_info as pk_person,
        tp3.julian_day, tp3.calendar
    from information.resource r0
    inner join projects.info_proj_rel ipr0 on ipr0.fk_entity = r0.pk_entity and ipr0.fk_project = {pk_project} and ipr0.is_in_project = true
    inner join information.statement s1 on s1.fk_subject_info = r0.pk_entity and s1.fk_property = {pks.properties.birth_broughtIntoLife_person}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_subject_info and s2.fk_property = {pks.properties.timeSpan_atSomeTimeWithin_timePrimitive}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.time_primitive tp3 on tp3.pk_entity = s2.fk_object_info
    where r0.fk_class = {pks.classes.birth}
""")
births['birthdate'] = [u.from_julian_day(row['julian_day'], row['calendar']) for _, row in births.iterrows()]
births['year'] = [date[0] if pd.notna(date) else pd.NA for date in births['birthdate']]
births['year'] = births['year'].astype(pd.Int16Dtype())
births.drop(columns=['julian_day', 'calendar'], inplace=True)

persons = persons.merge(births, how='left').drop_duplicates().reset_index(drop=True)

a.infos(persons)

Shape:  (25502, 4) - extract:


,pk_person,name,birthdate,year
0,25912,Odoardo Tabacchi,"(1831, 12, 19)",1831
1,25913,Gianfranco Miglio,"(1918, 1, 11)",1918
2,25914,Massimo Bontempelli,"(1878, 5, 12)",1878
3,25898,Giulio Bizzozero,"(1846, 3, 20)",1846
4,25899,Paul Jove,"(1483, 4, 19)",1483


# Find duplicates

In [34]:
matches = []
matches_key = set()

eta.begin(len(persons), "Findings duplicates")
for i, rowi in persons.iterrows():
    selection = persons[(rowi['year'] - 5 <= persons['year']) & (persons['year'] <= rowi['year'] + 5)]

    for j, rowj in selection.iterrows():
        if j < i: continue

        score = u.trigram_similarity(rowi['name'], rowj['name'])
        key = str(min(rowi['pk_person'], rowj['pk_person'])) + '-' + str(max(rowi['pk_person'], rowj['pk_person']))

        if score > 0.7 and rowi['pk_person'] != rowj['pk_person'] and key not in matches_key:
            matches.append({
                'pk_1': rowi['pk_person'],
                'name_1': rowi['name'],
                'birthdate_1': rowi['birthdate'],
                'pk_2': rowj['pk_person'],
                'name_2': rowj['name'],
                'birthdate_2': rowj['birthdate'],
            })
            # eta.print(f"Found similar:")
            # eta.print(f"    {rowi['pk_person']} {rowi['name']} {rowi['birthdate']}")
            # eta.print(f"    {rowj['pk_person']} {rowj['name']} {rowj['birthdate']}")
            matches_key.add(key)

    eta.iter()



Found similar:                                                                                                                   
    25941 Theodor Zwinger (1533, 8, 2)                                                                                           
    6516379 Theodor Zwinger (1533, 8, 12)                                                                                        
Found similar:                                                                                                                   
    26004 Hans Bühler (1893, 4, 12)                                                                                              
    10327148 Hans Bühler (1888, 2, 24)                                                                                           
Found similar:                                                                                                                   
    26087 Ami Argand (1750, 7, 5)                                                         